In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import numpy as np
import seaborn as sns
sns.set_theme()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
def log_mod(x):
    return np.log(abs(x))

In [ ]:
spx_price_day = pd.read_csv('data/SPX-Price-Daily.csv')
spx_price_minute = pd.read_csv('SPX_minute_2011_05_02-2022-02-18.csv')
spx_price_minute = spx_price_minute.drop(['ticker.1', 'trade_date.1'], axis = 1)

In [ ]:
spx_gamma = pd.read_csv('data/spx-gamma-dix.csv')

In [ ]:
# daily SPX Gamma distribution

In [ ]:
plt.hist(combined_day['gex'], bins = 50);

In [ ]:
# SPX Gamma exposure over time

In [ ]:
spx_gamma.set_index('date')['gex'].plot()

In [ ]:
spx_price_day.rename(columns={'pricing_date':'date'}, inplace=True)
spx_price_day.head()

In [ ]:
spx_price_minute.rename(columns={'trade_date':'date'}, inplace=True)
spx_price_minute.head()

In [ ]:
combined_day = pd.merge(spx_gamma,spx_price_day,on='date',how='inner')
combined_day['gex_scaled'] = pd.Series(combined_day['gex']/1e9).apply(lambda x: log_mod(x))
combined_day['volume_scaled'] = pd.Series(combined_day['volume']/1e6).apply(lambda x: log_mod(x)) - 2

In [ ]:
daily_liquidity = []
for i in range(len(combined_day)):
    if combined_day.iloc[i]['high'] == combined_day.iloc[i]['low']:
        daily_liquidity.append(combined_day.iloc[i]['volume_scaled']/abs(combined_day.iloc[i]['high'] - combined_day.iloc[i]['vwap']))
    else:
        daily_liquidity.append(combined_day.iloc[i]['volume_scaled']/abs(combined_day.iloc[i]['high'] - combined_day.iloc[i]['low']))

combined_day['liquidity'] = pd.Series(daily_liquidity) 

In [ ]:
# daily liquidity

In [ ]:
plt.hist(combined_day['liquidity'], bins = 25);

In [ ]:
# daily log(volume)

In [ ]:
plt.hist(combined_day['volume_scaled'], bins = 50);

In [ ]:
combined_minute =  pd.merge(spx_gamma,spx_price_minute,on='date',how='inner')
combined_minute['gex_scaled'] = pd.Series(combined_minute['gex']/1e9).apply(lambda x: log_mod(x))
combined_minute['volume'] = combined_minute['volume'].replace(0,np.mean(combined_minute['volume']))
combined_minute['volume_scaled'] = pd.Series(combined_minute['volume']/1e4).apply(lambda x: log_mod(x))

In [ ]:
from IPython.display import clear_output

In [ ]:
combined_minute['high'] = (combined_minute['high_bid_price'] + combined_minute['high_ask_price'])/2
combined_minute['low'] = (combined_minute['low_bid_price'] + combined_minute['low_ask_price'])/2
combined_minute['liquidity'] = combined_minute['volume_scaled']/abs(combined_minute['high'] - combined_minute['low'])/10
combined_minute['liquidity'] = combined_minute['liquidity'].replace(np.inf, np.mean(combined_minute['liquidity'])).replace(-np.inf, np.mean(combined_minute['liquidity']))
combined_minute['log_liquidity'] = combined_minute['liquidity'].apply(lambda x: log_mod(x))

In [ ]:
plt.hist(combined_minute['liquidity'], bins = 1000);
plt.xlim(0, 25)

In [ ]:
plt.hist(combined_minute['volume_scaled'], bins = 500);

In [ ]:
date_group = combined_minute.groupby('date').mean()[['gex', 'liquidity', 'gex_scaled', 'log_liquidity','high', 'low']]

In [ ]:
date_group['std_liquidity'] = [x for x in pd.Series(combined_minute.groupby('date').std()['liquidity'].to_list())]
date_group['std_log_liquidity'] = [x for x in pd.Series(combined_minute.groupby('date').std()['log_liquidity'].to_list())]

In [ ]:
plt.scatter(date_group['gex'], date_group['log_liquidity'])

In [ ]:
plt.scatter(date_group['gex'], date_group['std_log_liquidity'])

In [ ]:
date_group.index = pd.to_datetime(date_group.index)
pre_covid = date_group[date_group.index < '2020-03-01']
post_covid = date_group[date_group.index > '2020-03-01']

In [ ]:
plt.plot(pre_covid.groupby(pd.qcut(pre_covid['gex'],10)).mean()[['liquidity']]/3)
plt.xlabel("Gamma Bin")
plt.ylabel("Average Intraday Liquidity")
plt.title('Pre Covid SPX Gamma Vs Average Intraday Liquidity')

In [ ]:
plt.plot(post_covid.groupby(pd.qcut(post_covid['gex'],10)).mean()[['liquidity']])
plt.xlabel("Gamma Bin")
plt.ylabel("Average Intraday Liquidity")
plt.title('Post Covid SPX Gamma Vs Average Intraday Liquidity')

In [ ]:
combined_minute.to_csv('combined_minute.csv')